In [2]:
! pip install torch
!pip install transformers
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 5.3 MB/s eta 0:00:00


In [3]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU: Tesla T4


In [17]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, get_linear_schedule_with_warmup

tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-medium')
model = AutoModelWithLMHead.from_pretrained('tinkoff-ai/ruDialoGPT-medium')


dialogues_file_path = 'cutted_dial.txt'

train_dataset = TextDataset(tokenizer=tokenizer, file_path=dialogues_file_path, block_size=64)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir='./dialogue_fine_tuned',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=24,
    save_steps=1,
    save_total_limit=1,
    learning_rate=5e-4,
    gradient_accumulation_steps=1,
    dataloader_num_workers = 8,
    jit_mode_eval=True,
    fp16 = True
)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1468: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [19]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Step,Training Loss


Step,Training Loss


TrainOutput(global_step=49, training_loss=3.797814505440848, metrics={'train_runtime': 3736.5844, 'train_samples_per_second': 0.312, 'train_steps_per_second': 0.013, 'total_flos': 135474213814272.0, 'train_loss': 3.797814505440848, 'epoch': 1.0})

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
model.save_pretrained("./saved_model_medium")
tokenizer.save_pretrained("./saved_model_medium")


('./saved_model_medium/tokenizer_config.json',
 './saved_model_medium/special_tokens_map.json',
 './saved_model_medium/vocab.json',
 './saved_model_medium/merges.txt',
 './saved_model_medium/added_tokens.json',
 './saved_model_medium/tokenizer.json')

In [10]:
dialogues_file_path = 'dialogues.txt'

# Чтение файла с явной указанием кодировки
with open(dialogues_file_path, 'r', encoding='utf-8') as file:
    dialogues = file.read()
len(dialogues)

6881787

In [16]:
# text_to_write = "Это строка, которую вы хотите записать в файл."

# Указываем путь к файлу
file_path = "cutted_dial.txt"

# Открываем файл для записи (режим 'w' - write)
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(dialogues[:300125])

print("Строка успешно записана в файл.")

Строка успешно записана в файл.
